<a href="https://colab.research.google.com/github/QaziSaim/Fine-Tune-Projects/blob/main/Attention_Mechenisum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Luckly I already had saved my tokenizer now i am going to load it

In [12]:
import numpy as np
import pandas as pd
import tensorflow
import re
import string
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [13]:
df = pd.read_csv('/content/eng_to_fra.csv')

In [14]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(df.eng_texts)
eng_sequence = eng_tokenizer.texts_to_sequences(df.eng_texts)

In [15]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(df.fra_texts)
fra_sequence = fra_tokenizer.texts_to_sequences(df.fra_texts)

In [16]:
eng_vocab_size = len(eng_tokenizer.word_index) +1
fra_vocab_size = len(fra_tokenizer.word_index) + 1


In [17]:
max_eng_len = max(len(x) for x in eng_sequence)
max_fra_len = max(len(x) for x in fra_sequence)

In [18]:
encoder_input = pad_sequences(eng_sequence, maxlen=max_eng_len, padding='post')
decoder_input = pad_sequences([s[:-1] for s in fra_sequence],maxlen=max_fra_len-1,padding='post')
decoder_target = pad_sequences([s[1:] for s in fra_sequence],maxlen=max_fra_len-1,padding='post')

In [19]:
vocab_input_size = len(eng_tokenizer.word_index) + 1
vocab_target_size = len(fra_tokenizer.word_index) + 1

In [20]:
embedding = 128
units = 256
batch = 16
EPOCHES = 10

In [21]:
from tensorflow.keras.layers import Layer
class BahdanauAttetion(Layer):
  def __init__(self,units):
    super().__init__()
    self.w1 =  Dense(units)
    self.w2 = Dense(units)
    self.V = Dense(1)

  def call(self,query,value):
    query_with_time_axis = tensorflow.expand_dims(query, 1)
    score = self.V(tensorflow.nn.tanh(self.w1(value) + self.w2(query_with_time_axis)))
    attention_weights = tensorflow.nn.softmax(score, axis=1)
    context_vector = tensorflow.reduce_sum(attention_weights * value, axis=1)
    return context_vector, tensorflow.squeeze(attention_weights, -1)
